## NLP
NLP adalah cara bagi komputer untuk menganalisis, memahami, dan memperoleh makna dari bahasa manusia dengan cara yang cerdas dan bermanfaat. Dengan memanfaatkan NLP, pengembang dapat mengatur dan menyusun pengetahuan untuk melakukan tugas-tugas seperti peringkasan otomatis, terjemahan, pengenalan entitas bernama, ekstraksi hubungan, analisis sentimen, pengenalan ucapan, dan segmentasi topik.

## Import library yang diperlukan

In [1]:
import io #input output
import random #random numbers
import string # proses standard python strings
import warnings #untuk menampilkan warning ketika program eror
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer #TF (Term Frequency) adalah frekuensi dari kemunculan sebuah term dalam dokumen yang bersangkutan. Semakin besar jumlah kemunculan suatu term (TF tinggi) dalam dokumen, semakin besar pula bobotnya atau akan memberikan nilai kesesuaian yang semakin besar.
from sklearn.metrics.pairwise import cosine_similarity # Cosine similarity berfungsi untuk membandingkan kemiripan antar dokumen,
import warnings
warnings.filterwarnings('ignore')

## Download dan instal NLTK
NLTK (Natural Language Toolkit) adalah platform terkemuka untuk membangun program Python untuk bekerja dengan data bahasa manusia. Ini menyediakan antarmuka yang mudah digunakan ke lebih dari 50 sumber daya korpora dan leksikal seperti WordNet, bersama dengan rangkaian perpustakaan pemrosesan teks untuk klasifikasi, tokenisasi, stemming, penandaan, penguraian, dan penalaran semantik, pembungkus untuk perpustakaan NLP kekuatan industri.

[Natural Language Processing with Python](http://www.nltk.org/book/) 
(https://www.nltk.org/install.html)



In [2]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


### Install NLTK Package




In [3]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('popular', quiet=True) # untuk download package
nltk.download('punkt') # jika sudah pernah tidak usah di install lagi
nltk.download('wordnet') # jika sudah pernah tidak usah di install lagi

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Membaca database

Sebagai contoh, kami akan menggunakan halaman Wikipedia untuk chatbots sebagai database kami. Salin konten dari halaman dan letakkan di file teks bernama 'chatbot.txt'. Namun, Anda dapat menggunakan data base pilihan Anda.

In [4]:
f=open('chatbot.txt','r',errors = 'ignore')
raw=f.read()
raw = raw.lower()# converts to lowercase


Masalah utama dengan data teks adalah bahwa semuanya dalam format teks (string). Namun, algoritma pembelajaran mesin memerlukan semacam vektor fitur numerik untuk melakukan tugas tersebut. Jadi sebelum kita mulai dengan proyek NLP apa pun, kita perlu memprosesnya terlebih dahulu agar ideal untuk bekerja. Pra-pemrosesan teks dasar meliputi:

Mengubah seluruh teks menjadi huruf besar atau kecil, sehingga algoritma tidak memperlakukan kata yang sama dalam kasus yang berbeda

Tokenisasi: Tokenisasi hanyalah istilah yang digunakan untuk menggambarkan proses mengubah string teks normal menjadi daftar token, yaitu kata-kata yang sebenarnya kita inginkan. Tokenizer kalimat dapat digunakan untuk menemukan daftar kalimat dan tokenizer Word dapat digunakan untuk menemukan daftar kata dalam string.

Paket data NLTK mencakup tokenizer Punkt yang telah dilatih sebelumnya untuk bahasa Inggris.

Menghapus noise yaitu segala sesuatu yang tidak dalam angka atau huruf standar.
Menghapus kata Stop. Kadang-kadang, beberapa kata yang sangat umum yang tampaknya tidak terlalu berguna dalam membantu memilih dokumen yang sesuai dengan kebutuhan pengguna. Kata-kata ini disebut stopword
Stemming: Stemming adalah proses mereduksi kata-kata infleksi (atau kadang-kadang turunan) ke bentuk induk, dasar, atau akarnya — umumnya bentuk kata tertulis. Contoh jika kita melakukan stem pada kata-kata berikut: “Stems”, “Stemming”, “Stemmed”, “and Stemtization”, hasilnya akan menjadi satu kata “stem”.

Lemmatization: Sebuah varian kecil dari stemming adalah lemmatization. Perbedaan utama antara ini adalah, bahwa, stemming sering dapat membuat kata-kata yang tidak ada, sedangkan lemma adalah kata-kata yang sebenarnya. Contoh Lemmatisasi adalah bahwa “lari” adalah bentuk dasar dari kata-kata seperti “berlari” dan “lari” atau kata “lebih baik” dan “baik” berada dalam lemma yang sama sehingga dianggap sama.



## Tokenisasi

In [5]:
sent_tokens = nltk.sent_tokenize(raw)# mengubah ke kata berbentuk list
word_tokens = nltk.word_tokenize(raw)# mengubah kalimat ke list

## Preprocessing


In [6]:
lemmer = nltk.stem.WordNetLemmatizer()
#WordNet adalah kamus bahasa Inggris berorientasi semantik yang disertakan dalam NLTK.
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

## Mencocokkan keyword 

Selanjutnya, kita akan mendefinisikan fungsi untuk sapaan oleh bot yaitu jika input pengguna adalah sapaan, bot akan mengembalikan respons sapaan.

In [7]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
def greeting(sentence):
 
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

## Generating Response

### Bag of Words
Setelah fase prapemrosesan awal, kita perlu mengubah teks menjadi vektor angka yang bermakna. Bag-of-words adalah representasi teks yang menggambarkan kemunculan kata-kata dalam sebuah dokumen. Ini melibatkan dua hal:

* Sebuah kosakata yang dikenal.

* Ukuran keberadaan kata-kata yang dikenal.

Mengapa disebut "Bag of words" ? Itu karena semua informasi tentang urutan atau struktur kata dalam dokumen dibuang dan model hanya **memperhatikan apakah kata yang diketahui muncul di dokumen**

Misalnya, jika kamus kami berisi kata-kata {Learning, is, the, not, great}, dan kami ingin membuat vektor teks “Learning is great”, kami akan memiliki vektor berikut: (1, 1, 0, 0, 1).


### Pendekatan TF-IDF 
Masalah dengan pendekatan Bag of Words adalah bahwa kata-kata yang sangat sering mulai mendominasi dalam dokumen (misalnya skor yang lebih besar), tetapi mungkin tidak mengandung banyak "konten informasi".

Salah satu pendekatannya adalah dengan menskalakan ulang frekuensi kata berdasarkan seberapa sering kata tersebut muncul di semua dokumen sehingga skor untuk kata yang sering seperti "the" yang juga sering muncul di semua dokumen akan dikenakan sanksi. Pendekatan untuk penilaian ini disebut Term Frequency-Inverse Document Frequency, atau disingkat TF-IDF, di mana:

**Frekuensi Istilah: adalah penilaian frekuensi kata dalam dokumen.**

```
TF = (Jumlah istilah t muncul dalam dokumen)/(Jumlah istilah dalam dokumen)
```

**Frekuensi Dokumen Terbalik: adalah penilaian seberapa jarang kata tersebut di seluruh dokumen.**

```
IDF = 1+log(N/n), di mana, N adalah jumlah dokumen dan n adalah jumlah dokumen tempat t muncul.
```
### Cosine Similarity

Bobot tf-idf adalah bobot yang sering digunakan dalam pencarian informasi dan penambangan teks. Bobot ini adalah ukuran statistik yang digunakan untuk mengevaluasi seberapa penting sebuah kata bagi dokumen dalam kumpulan atau database

```
Cosine Similarity (d1, d2) = Hasil kali titik(d1, d2) / ||d1|| * ||d2||
```
di mana d1,d2 adalah dua vektor bukan nol.


Untuk menghasilkan tanggapan dari bot, konsep kesamaan dokumen akan digunakan. Kami mendefinisikan respons fungsi yang mencari ucapan pengguna untuk satu atau lebih kata kunci yang dikenal dan mengembalikan salah satu dari beberapa kemungkinan respons. Jika tidak menemukan input yang cocok dengan salah satu kata kunci, ia mengembalikan respons: "Maaf! aku tidak mengerti kamu”

In [8]:
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response



kami ingin bot kami memunculkan kata saat memulai dan mengakhiri percakapan tergantung pada input pengguna.

In [9]:
flag=True
print("ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("ROBO: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("ROBO: "+greeting(user_response))
            else:
                print("ROBO: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ROBO: Bye! take care..")

ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!
chatbots
ROBO: in 2016, facebook messenger allowed developers to place chatbots on their platform.
background
ROBO: background
in 1950, alan turing's famous article "computing machinery and intelligence" was published, which proposed what is now called the turing test as a criterion of intelligence.
bye
ROBO: Bye! take care..
